In [1]:
import pandas as pd


In [ ]:
raw_oc_release = pd.read_excel('Product Release 2.1.xlsx', header=0)  # specify the actual sheet name
oc_release = raw_oc_release.iloc[1:]
oc_release.head(3)

In [ ]:
# Split the 'second_column' by new line
oc_release['[SA_only] Link to Use Case'] = oc_release['[SA_only] Link to Use Case'].str.split('\n')
pd.set_option('display.max_colwidth', None)
oc_release.head(3)

In [ ]:
for index, row in oc_release.iterrows():
    print(row['[SA_only] Link to Use Case'])

In [ ]:
# Use explode to duplicate the first column and expand the second column
df_exploded = oc_release.explode('[SA_only] Link to Use Case')

df_exploded.head(5)

In [ ]:
# Replace text in the column using regular expression to extract the pattern U-xxxxx
df_exploded['[SA_only] Link to Use Case'] = df_exploded['[SA_only] Link to Use Case'].str.extract(r'(U-\d+)')
oc_release_exploded = df_exploded
oc_release_exploded.head(5)


In [ ]:
# Load the Excel files into dataframes
raw_fl_release = pd.read_excel('FL Release 2.1.1.xlsx', header=0)  # specify the actual sheet name
raw_fl_release.head(3)

In [ ]:
print(oc_release_exploded["[SA_only] Link to Use Case"])

In [ ]:
print(raw_fl_release["UID"])

In [ ]:
column_dtype = oc_release_exploded["[SA_only] Link to Use Case"].dtype
print(column_dtype)

In [ ]:
column_dtype = raw_fl_release["UID"].dtype
print(column_dtype)

In [ ]:
# Convert Series to sets and then use the difference operation
unmatched_flucs_in_raw_fl_release = set(oc_release_exploded["[SA_only] Link to Use Case"]).difference(set(raw_fl_release["UID"]))

# Output the UIDs
print("UIDs present in the devprom but not in the release manifest:")
print(list(unmatched_flucs_in_raw_fl_release))

In [ ]:
# Convert raw_fl_release["UID"] to a set for faster lookup
release_uids_set = set(raw_fl_release["UID"])

# Boolean index for unmatched UIDs in oc_release_exploded
unmatched_mask = ~oc_release_exploded["[SA_only] Link to Use Case"].isin(release_uids_set)

# Get the rows with unmatched UIDs
unmatched_rows = oc_release_exploded[unmatched_mask]

# Display the unmatched rows
print("Rows in oc_release_exploded with UIDs not present in raw_fl_release:")
unmatched_rows

In [ ]:
# move [SA_only] Link to Use Case as the first column in unmatched_rows 
#unmatched_rows = unmatched_rows[["[SA_only] Link to Use Case"] + list(unmatched_rows.columns[:-1])]
unmatched_rows

In [ ]:
unmatched_rows["[SA_only] Link to Use Case"]

In [ ]:
unique_uids = unmatched_rows["UID"].unique()

# Convert the array of unique UIDs into a comma-separated string
unique_uids_str = ', '.join(unique_uids)

print("Unique UIDs in unmatched_rows:")
print("# of UCs:", len(unique_uids), unique_uids_str)

In [54]:
#export to excel
unmatched_rows.to_excel("unmatched fl ucs from oc release in fl release.xlsx")